# Review Summarization Prompt Optimization with DSPy (SIMBA/MIPROv2)

This script demonstrates how to optimize prompts in a DSPy program for summarizing user reviews.

## References:
- https://dspy.ai/learn/optimization/optimizers/#__tabbed_1_3
- https://dspy.ai/tutorials/classification_finetuning/
- https://dspy.ai/tutorials/math/

In [9]:
# 1. Setup
# Import required libraries and configure the language model.
import dspy
from typing import List, Dict
from app.llm.review_summarizer import SummarizeSignature, ReviewSummarizer
from app.llm.language import SupportedLanguage
import os

# Configure the language model (replace with your preferred model)
# dspy.configure(lm=dspy.LM('gpt-4.1-nano'))

In [10]:
summarizer = ReviewSummarizer()

# Test the Optimized Program
test_reviews = [
    "The app is easy to use and very intuitive.",
    "Customer support was quick to respond.",
    "Some features are missing compared to competitors."
]

# Test Czech
czech_result = summarizer(
    reviews="\n".join(test_reviews),
    language=SupportedLanguage.CZECH
)
print("Czech Summary:\n", czech_result.summary.replace(".", ".\n"))

Czech Summary:
 Objevte jednoduchost a intuitivnost naší aplikace, která vám umožní snadno a rychle zvládnout vaše úkoly.
 Díky naší rychlé a ochotné zákaznické podpoře získáte vždy potřebnou pomoc.
 Přestože některé funkce chybí ve srovnání s konkurencí, naše řešení nabízí skvělý základ pro vaše každodenní použití a neustále se vyvíjí, aby vám přineslo ještě více hodnoty.



In [11]:
# 2. Prepare Training Data
trainset = [
    # Hotel reviews in Czech
    dspy.Example(
        reviews=[
            "Room was clean and spacious.",
            "Excellent location, but noisy at night.",
            "Staff was helpful and check-in was quick."
        ],
        language="cs",  # Simple string
        reference="Uživatelé oceňují čistotu hotelu. Přilehlé ulice jsou hlučné."
    ).with_inputs("reviews", "language"),
    
    # Hotel reviews in Slovak
    dspy.Example(
        reviews=[
            "Room was clean and spacious.",
            "Excellent location, but noisy at night.",
            "Staff was helpful and check-in was quick."
        ],
        language="sk",  # Simple string
        reference="Používatelia oceňujú čistotu hotela. Priľahlé ulice sú hlučné."
    ).with_inputs("reviews", "language"),
    
    # Product reviews in Czech
    dspy.Example(
        reviews=[
            "Battery life is impressive.",
            "Screen quality is not as good as expected.",
            "Affordable price for the features offered."
        ],
        language="cs",  # Simple string
        reference="Uživatelé oceňují dlouhou výdrž baterie. Kvalita obrazovky by mohla být lepší."
    ).with_inputs("reviews", "language"),
    
    # Product reviews in Slovak
    dspy.Example(
        reviews=[
            "Battery life is impressive.",
            "Screen quality is not as good as expected.",
            "Affordable price for the features offered."
        ],
        language="sk",  # Simple string
        reference="Používatelia oceňujú dlhú výdrž batérie. Kvalita obrazovky by mohla byť lepšia."
    ).with_inputs("reviews", "language")
]

In [12]:
from typing import Literal

# 4. Define LLM as Judge Metric
class JudgeSignature(dspy.Signature):
    """Judge if the summary is a good, faithful, and persuasive summary of the reviews in the specified language."""
    reviews = dspy.InputField()
    language = dspy.InputField(desc="Language code (cs = czech or sk = slovak)")
    reference = dspy.InputField()
    prediction = dspy.InputField()
    score: Literal[0,1] = dspy.OutputField(
        desc="1 if the prediction is a good, faithful, and persuasive summary of the reviews in the specified language, 0 otherwise."
    )

llm_judge = dspy.ChainOfThought(JudgeSignature)


def llm_judge_metric(example, prediction, trace=None):
    """Judge if the generated summary matches the reference in the specified language."""
    result = llm_judge(
        reviews="\n".join(example.reviews),
        language=example.language,  # Just pass the string directly
        reference=example.reference,
        prediction=prediction.summary
    )
    return float(result.score)

In [14]:
# 5. Run the Optimizer (SIMBA or MIPROv2)
optimizer = dspy.MIPROv2(
    metric=llm_judge_metric,
    num_threads=4,
    max_bootstrapped_demos=2,
    auto="light"
)

# Compile (optimize) the program
optimized_summarizer = optimizer.compile(
    ReviewSummarizer(),
    trainset=trainset,
    requires_permission_to_run=False
)

2025/05/26 08:49:09 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 3

2025/05/26 08:49:09 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/26 08:49:09 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/26 08:49:09 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


2025/05/26 08:49:09 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
100%|██████████| 1/1 [00:00<00:00, 241.97it/s]


Bootstrapped 0 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.
Bootstrapping set 4/6


2025/05/26 08:49:09 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
100%|██████████| 1/1 [00:00<00:00, 302.86it/s]


...

2025/05/26 08:50:08 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


  0%|          | 0/3 [00:00<?, ?it/s]

2025/05/26 08:50:08 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:00<00:00, 1338.04it/s]

2025/05/26 08:50:08 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/05/26 08:50:08 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 100.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 3', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 1'].
2025/05/26 08:50:08 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [100.0, 100.0, 100.0, 100.0, 66.67, 100.0, 100.0, 100.0, 100.0, 66.67, 100.0, 100.0, 66.67, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
2025/05/26 08:50:08 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 100.0
2025/05/26 08:50:08 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/05/26 08:50:08 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 100.0!


In [19]:
# 4. Test the Optimized Program
test_reviews = [
    "The app is easy to use and very intuitive.",
    "Customer support was quick to respond.",
    "Some features are missing compared to competitors."
]

# Test Czech
czech_result = optimized_summarizer(
    reviews="\n".join(test_reviews),
    language=SupportedLanguage.CZECH
)
print("Czech Summary:", czech_result.summary)

# Test Slovak
slovak_result = optimized_summarizer(
    reviews="\n".join(test_reviews),
    language=SupportedLanguage.SLOVAK
)
print("\nSlovak Summary:", slovak_result.summary)

Czech Summary: Objevte jednoduchost a intuitivnost naší aplikace, která vám umožní snadno a rychle zvládnout vaše úkoly. Díky naší rychlé a ochotné zákaznické podpoře získáte vždy potřebnou pomoc. Přestože některé funkce chybí ve srovnání s konkurencí, naše řešení nabízí skvělý základ pro vaše každodenní použití a neustále se vyvíjí, aby vám přineslo ještě více hodnoty.

Slovak Summary: Objavte jednoduchú a intuitívnu aplikáciu, ktorá vám uľahčí každodenné úlohy. Rýchla a efektívna podpora zákazníkov vám zabezpečí pokoj na duši. Hoci niektoré funkcie chýbajú v porovnaní s konkurenciou, jej jednoduché ovládanie a spoľahlivosť ju robia skvelou voľbou pre tých, ktorí hľadajú efektívne riešenie s ľahkým používaním.


In [20]:
dspy.inspect_history(n=2)





[2025-05-26T08:52:54.870322]

System message:

Your input fields are:
1. `reviews` (str): All user reviews as a markdown list.
2. `language` (str): The language code for the output summary (cs = czech or sk = slovak).
Your output fields are:
1. `summary` (str): A factual summary of user reviews.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## reviews ## ]]
{reviews}

[[ ## language ## ]]
{language}

[[ ## summary ## ]]
{summary}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Summarize the provided user reviews.
        # Role
        You are an expert review summarizer. Your job is to create top-notch summaries of user reviews.
        # Instructions
        - Summarize the provided reviews
        - Extract key information about what users like, dislike and potential idiosyncracies of the product that users reported
        - Provide the summary in the specified language


User message:

[[ 

In [26]:
# Find the latest version number
import glob
import re


# 7. Save and Load the Optimized Program
# Ensure the output directory exists
output_dir = os.path.abspath('experiments/optimized_prompts')
os.makedirs(output_dir, exist_ok=True)

# Get all existing versioned files
existing_files = glob.glob(os.path.join(output_dir, "v*-optimized_summarizer.json"))
version_numbers = []

for file in existing_files:
    match = re.search(r'v(\d+)', file)
    if match:
        version_numbers.append(int(match.group(1)))

# Determine next version number
next_version = max(version_numbers) + 1 if version_numbers else 1

# Create new filename with incremented version
output_path = os.path.join(output_dir, f"v{next_version}-optimized_summarizer.json")

optimized_summarizer.save(output_path)

loaded_summarizer = ReviewSummarizer()
loaded_summarizer.load(path=output_path)